In [11]:
import os
import sys

# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

# Now you can import the module from the examples directory
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.sparse import csc_matrix
from jax import vmap
import networkx as nx
from functools import partial
from scipy.spatial import distance_matrix
from l2ws.utils.nn_utils import get_nearest_neighbors
from l2ws.l2ws_model import L2WSmodel
from l2ws.scs_model import SCSmodel
from l2ws.algo_steps import create_k_steps_train, create_k_steps_eval, create_M, get_scaled_vec_and_factor, create_projection_fn


## SCS

In this tutorial we show how to use our package to learn high-quality warm starts for your favorite fixed-point algorithm.
We consider projected gradient descent to solve the problem

\begin{array}{ll} \text{minimize} & 1 / 2 x^T P x + c^T x
\\ \text{subject to} \quad & Ax + s = b\\
&s \in \mathcal{K}
 \end{array}
where $x \in \mathbf{R}^n$ and $s \in \mathbf{R}^m$ are the decision variables.
Here, $P \in \mathbf{S}_+^n$, $A \in \mathbf{R}^{m \times n}$, $c \in \mathbf{R}^n$, and $b \in \mathbf{R}^m$ are the problem data.


We take the specific example of maxcut

\begin{array}{ll} \rm{minimize} &\mathbf{tr}(CX) \\
&X_{ii} = 1 \quad i=1,\dots,n \\
&X \succeq 0
 \end{array}
where $C \in \mathbf{S}^{n}_+$ is the problem data.
Here, $\theta = \mathbf{vec}(C)$ is the parameter.

In [12]:
# # generate the problem data that does not change
# # in this case, the matrices P and A do not change from problem to problem
# # therefore the matrix factorization is the same for all problems
# n = 20
# nc2 = int(n * (n + 1) / 2)
# m = n
# P = jnp.zeros((n, n))
# A = jnp.zeros((m, nc2))
# b = jnp.ones(m)
# cones = dict(z=m, )

# # basic hyperparameters in SCS
# rho_x, alpha, scale = 1, 1, 1

# # the part of \mathcal{K} that is the zero cone
# zero_cone_size = m

# M = create_M(P, A)
# factor = get_scaled_vec_and_factor(M, rho_x, scale, m, nc2, zero_cone_size)

# projection = create_projection_fn(cones, nc2)



In [23]:
# create the cvxpy problem
n_orig = 20
nc2 = int(n_orig * (n_orig + 1) / 2)
N_train = 100
N_test = 10
N = N_train + N_test

C_param = cp.Parameter((n_orig, n_orig), symmetric=True)
X = cp.Variable((n_orig, n_orig), symmetric=True)
prob = cp.Problem(cp.Maximize(cp.trace(C_param @ X)), constraints=[X >> 0, cp.diag(X) == 1])

In [26]:
# create the C matrices
laplacian_matrices = []
laplacian_matrices_upper_tri_mat = np.zeros((N, nc2))

for i in range(N):
    # Create a random graph (you can customize the graph generation method)
    G = nx.erdos_renyi_graph(n_orig, 0.5)  # Example: Erdos-Renyi random graph
    
    # Compute the Laplacian matrix of the graph
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    
    laplacian_matrices.append(laplacian_matrix)
    laplacian_matrices_upper_tri_mat[i, :] = laplacian_matrix[np.triu_indices(laplacian_matrix.shape[0])]

In [27]:
C_param.value = laplacian_matrices[0]
prob.solve()
data, _, __ = prob.get_problem_data(cp.SCS)
c, b = data['c'], data['b']
A = data['A']

m = b.size
n = c.size
P = csc_matrix(np.zeros((n, n)))

cones_cp = data['dims']
cones = {'z': cones_cp.zero, 'l': cones_cp.nonneg, 'q': cones_cp.soc, 's': cones_cp.psd}

q_mat = np.zeros((N, m + n))
z_stars = np.zeros((N, m + n))
for i in range(N):
    C_param.value = laplacian_matrices[i]
    sol = prob.solve()
    data, _, __ = prob.get_problem_data(cp.SCS)
    c, b = data['c'], data['b']

    # q = (c, b)
    q_mat[i, :n] = c
    q_mat[i, n:] = b

    # transform the solution to the z variable
    z = sol.x


In [28]:
# SCS algorithm hyperparameters
rho_x, alpha, scale = 1, 1, 1
zero_cone_size = cones['z']
P_jax = jnp.array(P.todense())
A_jax = jnp.array(A.todense())
M_jax = create_M(P_jax, A_jax)
algo_factor, scale_vec = get_scaled_vec_and_factor(M_jax, rho_x, scale, m, n,
                                                        zero_cone_size)

In [29]:
q_mat_jax = jnp.array(q_mat)
theta_mat_jax = jnp.array(laplacian_matrices_upper_tri_mat)


In [ ]:
# generate the different C matrices

In [33]:
projection = create_projection_fn(cones, n)
q_mat_train = q_mat_jax[:N_train, :]
q_mat_test = q_mat_jax[N_train:, :]
train_inputs = theta_mat_jax[:N_train, :]
test_inputs = theta_mat_jax[N_train:, :]

input_dict = dict(algorithm='scs',
                      m=m, n=n, hsde=True, static_flag=True, proj=projection,
                      cones=cones,
                      train_unrolls=5, jit=True,
                      q_mat_train=q_mat_train, q_mat_test=q_mat_test,
                      train_inputs=train_inputs, test_inputs=test_inputs,
                      static_M=M_jax,
                      static_algo_factor=algo_factor,
                      rho_x=rho_x, scale=scale, alpha_relax=alpha,
                      zero_cone_size=zero_cone_size)
scs_model = SCSmodel(input_dict)

In [34]:
# cold start evaluation
k = 100
init_eval_out = scs_model.evaluate(
    k, test_inputs, q_mat_test, z_stars=z_stars_test, fixed_ws=False, tag='test')
init_test_losses = init_eval_out[1][1].mean(axis=0)

NameError: name 'proj_gd_model' is not defined